# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0902 22:36:43.947000 944366 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:36:43.947000 944366 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:36:44] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', tokenizer_worker_num=1, skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30881, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, s

[2025-09-02 22:36:45] Using default HuggingFace chat template with detected content format: string


W0902 22:36:52.967000 945275 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:36:52.967000 945275 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0902 22:36:53.064000 945276 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:36:53.064000 945276 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:36:53] `torch_dtype` is deprecated! Use `dtype` instead!


...

Capturing batches (bs=1 avail_mem=53.66 GB):  75%|███████▌  | 3/4 [00:06<00:01,  1.53s/it]

Capturing batches (bs=1 avail_mem=53.66 GB): 100%|██████████| 4/4 [00:10<00:00,  2.52s/it]
[2025-09-02 22:37:13] Capture draft cuda graph end. Time elapsed: 10.82 s. mem usage=-21.01 GB. avail mem=53.62 GB.
[2025-09-02 22:37:13] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=53.62 GB


Capturing batches (bs=2 avail_mem=53.07 GB):  50%|█████     | 2/4 [00:00<00:00, 15.11it/s]

Capturing batches (bs=1 avail_mem=53.05 GB): 100%|██████████| 4/4 [00:00<00:00, 18.46it/s]
[2025-09-02 22:37:15] Capture draft extend cuda graph end. Time elapsed: 1.64 s. mem usage=0.57 GB. avail mem=53.05 GB.
[2025-09-02 22:37:15] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=53.05 GB
[2025-09-02 22:37:15] INFO:     Started server process [944366]
[2025-09-02 22:37:15] INFO:     Waiting for application startup.
[2025-09-02 22:37:15] INFO:     Application startup complete.
[2025-09-02 22:37:15] INFO:     Uvicorn running on http://127.0.0.1:30881 (Press CTRL+C to quit)


[2025-09-02 22:37:16] INFO:     127.0.0.1:41826 - "GET /v1/models HTTP/1.1" 200 OK
[2025-09-02 22:37:16] INFO:     127.0.0.1:41832 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-09-02 22:37:16] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-02 22:37:16] INFO:     127.0.0.1:41834 - "POST /generate HTTP/1.1" 200 OK
[2025-09-02 22:37:16] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-09-02 22:37:21] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-02 22:37:21] INFO:     127.0.0.1:41842 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0902 22:37:29.662000 948358 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:37:29.662000 948358 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-09-02 22:37:30] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', tokenizer_worker_num=1, skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=36869, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=9

[2025-09-02 22:37:31] Using default HuggingFace chat template with detected content format: string


W0902 22:37:38.474000 949055 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:37:38.474000 949055 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0902 22:37:38.474000 949054 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:37:38.474000 949054 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:37:39] `torch_dtype` is deprecated! Use `dtype` instead!


...

Capturing batches (bs=1 avail_mem=17.78 GB): 100%|██████████| 4/4 [00:16<00:00,  4.13s/it]
[2025-09-02 22:38:41] Capture draft cuda graph end. Time elapsed: 17.26 s. mem usage=0.95 GB. avail mem=36.27 GB.
[2025-09-02 22:38:41] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=36.27 GB


Capturing batches (bs=1 avail_mem=36.18 GB): 100%|██████████| 4/4 [00:00<00:00, 47.00it/s]
[2025-09-02 22:38:42] Capture draft extend cuda graph end. Time elapsed: 0.81 s. mem usage=0.10 GB. avail mem=36.17 GB.
[2025-09-02 22:38:42] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=36.17 GB


[2025-09-02 22:38:42] INFO:     Started server process [948358]
[2025-09-02 22:38:42] INFO:     Waiting for application startup.
[2025-09-02 22:38:42] INFO:     Application startup complete.
[2025-09-02 22:38:42] INFO:     Uvicorn running on http://127.0.0.1:36869 (Press CTRL+C to quit)


[2025-09-02 22:38:43] INFO:     127.0.0.1:35878 - "GET /v1/models HTTP/1.1" 200 OK
[2025-09-02 22:38:43] INFO:     127.0.0.1:35884 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-09-02 22:38:43] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:38:45] INFO:     127.0.0.1:35886 - "POST /generate HTTP/1.1" 200 OK
[2025-09-02 22:38:45] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-09-02 22:38:48] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:38:48] INFO:     127.0.0.1:36742 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0902 22:38:55.977000 957904 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:38:55.977000 957904 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-09-02 22:38:57] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', tokenizer_worker_num=1, skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34602, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False

[2025-09-02 22:38:57] Using default HuggingFace chat template with detected content format: string


W0902 22:39:06.115000 958540 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:39:06.115000 958540 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0902 22:39:06.122000 958541 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:39:06.122000 958541 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:39:06] `torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:39:06] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=4.60 GB):  75%|███████▌  | 3/4 [00:04<00:01,  1.05s/it]

Capturing batches (bs=1 avail_mem=4.60 GB): 100%|██████████| 4/4 [00:06<00:00,  1.58s/it]
[2025-09-02 22:39:34] Capture draft cuda graph end. Time elapsed: 6.83 s. mem usage=0.29 GB. avail mem=4.54 GB.
[2025-09-02 22:39:34] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=4.54 GB


Capturing batches (bs=1 avail_mem=4.44 GB): 100%|██████████| 4/4 [00:00<00:00, 31.26it/s]
[2025-09-02 22:39:35] Capture draft extend cuda graph end. Time elapsed: 0.87 s. mem usage=0.10 GB. avail mem=4.44 GB.
[2025-09-02 22:39:35] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=4.44 GB


[2025-09-02 22:39:35] INFO:     Started server process [957904]
[2025-09-02 22:39:35] INFO:     Waiting for application startup.
[2025-09-02 22:39:35] INFO:     Application startup complete.
[2025-09-02 22:39:35] INFO:     Uvicorn running on http://127.0.0.1:34602 (Press CTRL+C to quit)
[2025-09-02 22:39:36] INFO:     127.0.0.1:45994 - "GET /v1/models HTTP/1.1" 200 OK


[2025-09-02 22:39:36] INFO:     127.0.0.1:46008 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-09-02 22:39:36] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-09-02 22:39:37] INFO:     127.0.0.1:46012 - "POST /generate HTTP/1.1" 200 OK
[2025-09-02 22:39:37] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-09-02 22:39:41] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:39:41] INFO:     127.0.0.1:46018 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0902 22:39:49.428000 961403 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:39:49.428000 961403 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-09-02 22:39:50] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', tokenizer_worker_num=1, skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=31960, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-09-02 22:39:51] Using default HuggingFace chat template with detected content format: string


W0902 22:39:58.639000 962081 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:39:58.639000 962081 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0902 22:39:58.639000 962080 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:39:58.639000 962080 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:39:59] `torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:39:59] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=38.62 GB): 100%|██████████| 4/4 [00:00<00:00, 52.42it/s]
[2025-09-02 22:40:24] Capture draft extend cuda graph end. Time elapsed: 0.63 s. mem usage=0.11 GB. avail mem=38.62 GB.
[2025-09-02 22:40:24] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=38.62 GB


[2025-09-02 22:40:25] INFO:     Started server process [961403]
[2025-09-02 22:40:25] INFO:     Waiting for application startup.
[2025-09-02 22:40:25] INFO:     Application startup complete.
[2025-09-02 22:40:25] INFO:     Uvicorn running on http://127.0.0.1:31960 (Press CTRL+C to quit)


[2025-09-02 22:40:25] INFO:     127.0.0.1:42754 - "GET /v1/models HTTP/1.1" 200 OK


[2025-09-02 22:40:26] INFO:     127.0.0.1:32842 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-09-02 22:40:26] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:40:26] INFO:     127.0.0.1:32858 - "POST /generate HTTP/1.1" 200 OK
[2025-09-02 22:40:26] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-09-02 22:40:30] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:40:31] INFO:     127.0.0.1:32864 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../basic_usage/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0902 22:40:38.441000 964277 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:40:38.441000 964277 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-09-02 22:40:39] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', tokenizer_worker_num=1, skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=37685, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=556851499, constrain

`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-02 22:40:40] `torch_dtype` is deprecated! Use `dtype` instead!


[2025-09-02 22:40:40] Using default HuggingFace chat template with detected content format: string


W0902 22:40:47.425000 964841 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0902 22:40:47.425000 964841 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


...

Capturing batches (bs=1 avail_mem=42.69 GB): 100%|██████████| 4/4 [00:00<00:00, 24.02it/s]
[2025-09-02 22:41:01] Capture draft extend cuda graph end. Time elapsed: 0.80 s. mem usage=0.31 GB. avail mem=42.69 GB.
[2025-09-02 22:41:01] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=42.69 GB


[2025-09-02 22:41:01] INFO:     Started server process [964277]
[2025-09-02 22:41:01] INFO:     Waiting for application startup.
[2025-09-02 22:41:01] INFO:     Application startup complete.
[2025-09-02 22:41:01] INFO:     Uvicorn running on http://0.0.0.0:37685 (Press CTRL+C to quit)


[2025-09-02 22:41:02] INFO:     127.0.0.1:52774 - "GET /v1/models HTTP/1.1" 200 OK


[2025-09-02 22:41:02] INFO:     127.0.0.1:52790 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-09-02 22:41:02] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:41:03] INFO:     127.0.0.1:52792 - "POST /generate HTTP/1.1" 200 OK
[2025-09-02 22:41:03] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-09-02 22:41:07] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-09-02 22:41:09] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: True, gen throughput (token/s): 9.01, #queue-req: 0, 


[2025-09-02 22:41:11] Decode batch. #running-req: 1, #token: 155, token usage: 0.01, accept len: 1.60, cuda graph: True, gen throughput (token/s): 31.59, #queue-req: 0, 


[2025-09-02 22:41:11] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, accept len: 1.73, cuda graph: True, gen throughput (token/s): 179.21, #queue-req: 0, 


[2025-09-02 22:41:12] Decode batch. #running-req: 1, #token: 293, token usage: 0.01, accept len: 1.73, cuda graph: True, gen throughput (token/s): 178.46, #queue-req: 0, 


[2025-09-02 22:41:12] Decode batch. #running-req: 1, #token: 362, token usage: 0.02, accept len: 1.73, cuda graph: True, gen throughput (token/s): 175.46, #queue-req: 0, 


[2025-09-02 22:41:12] Decode batch. #running-req: 1, #token: 430, token usage: 0.02, accept len: 1.70, cuda graph: True, gen throughput (token/s): 177.30, #queue-req: 0, 
[2025-09-02 22:41:12] INFO:     127.0.0.1:49128 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).